In [116]:
import numpy as np

In [1119]:
# Import dataframe into MySQL
import sqlalchemy
def etl_mysql():
    database_username = 'root'
    database_password = 'rmit2001'
    database_ip       = 'localhost'
    database_name     = 'LINKEDIN_USER'
    database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                                   format(database_username, database_password, 
                                                          database_ip, database_name))
    return database_connection

In [1114]:
university_map = 'https://stackoverflow.com/questions/53071949/fuzzy-matching-join-two-data-frames-of-university-names'

In [434]:
def estimate(A,B):
    count = 0
    for i in range(len(A)):
        if(A[i] != 0):
            if(A[i] != B[i]):  # A1 B0
                count = count+1
    return 1 - count/len(A)

In [446]:
a = [0]*817

In [452]:
X['matching_score'] = X.apply(lambda row : estimate(a,row), axis = 1)

In [462]:
max_index_row = np.argmax(X['matching_score'])

In [474]:
df_.columns

Index(['ID', 'management', 'project management', 'javascript', 'java',
       'leadership', 'html', 'cloud computing', 'sql', 'microsoft office',
       ...
       'arcsight', 'symantec endpoint protection', 'security training', 'wsus',
       'soar', 'soc', 'malware analysis', 'metasploit', 'antivirus', 'ccnp'],
      dtype='object', length=830)

,management,project management,javascript,java,leadership,html,cloud computing,sql,microsoft office,software development,...,arcsight,symantec endpoint protection,security training,wsus,soar,malware analysis,metasploit,antivirus,ccnp,matching_score
0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.998776
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.993880
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.995104
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.993880
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.993880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.993880
870,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.993880
871,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.993880
872,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.993880


In [697]:
estimate(X.iloc[0],X.iloc[1])

0.9706601466992665

In [755]:
# skill, location, current_company, education 

In [754]:
from scipy import spatial
List1 = [4, 47, 8, 3]
List2 = [3, 52, 12, 16]
result = 1 - spatial.distance.cosine(List1, List2)
print(result)

0.9720951480078084


In [1115]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["linkedin_mentor"]
df = pd.DataFrame(list(mycol.find({}, {'_id' : 0})))

## Preprocessing Experience

In [1116]:
#some company name is in list type, it's related to the scraping error with noise like : [2y3r , fulltime]. The solution is take the Jobtitle as the Company Name
def preprocessing_experience(row):
    if(len(row.Experience) and type(row.Experience['0']['Company']) == list):
            row.Experience['0']['Company'] = row['Job Title']
            row.experience_list[0] = row.Experience['0']['Company']
    return row
for i  in range(len(df)):
    preprocessing_experience(df.iloc[i])

In [1117]:
#take the first company as current company
def current_company(row):
    if(len(row.experience_list) > 0):
        return row.experience_list[0]
df['current_company']=df.apply(lambda row : current_company(row), axis = 1)

In [1120]:
company_query = "Select * from Companies"
companies = pd.read_sql(company_query, conn)

In [1121]:
#!pip3 install fuzzywuzzy
from fuzzywuzzy import fuzz
import pandas as pd
import numpy as np

In [1122]:
def partial_match(x,y):
    return(fuzz.partial_ratio(Str1.lower(),Str2.lower()))
partial_match_vector = np.vectorize(partial_match)

In [1123]:
companies['Key'] = 1
df['Key'] = 1
combined_dataframe = df.merge(companies,on="Key",how="left")

In [1126]:
def fuzzy_match(row):
    Str1 = str(row.current_company)
    Str2 = str(row.Company)
    return fuzz.partial_ratio(Str1.lower(),Str2.lower())

In [1127]:
combined_dataframe['score']=combined_dataframe.apply(lambda row : fuzzy_match(row), axis = 1)
combined_dataframe = combined_dataframe[combined_dataframe.score>=80]

In [1128]:
final = combined_dataframe.drop(columns = {'Key', 'index', 'score'})

In [1131]:
final.set_index(final['id']).drop(columns = {'id', 'experience_list', 'current_company'})

,Name,Job Title,Location,Image,About,Education,Experience,Skill,Honors & awards,Publication,Languages,school_list,skill_list,username,password,City,Company
id,,,,,,,,,,,,,,,,,
1,Thanh Nguyen,Senior Solutions Architect at Amazon Web Servi...,"Ho Chi Minh City, Vietnam",[],"• With 20+ years of global experience, DevOps ...","{'0': {'Major': 'Master of Science - MS, Compu...",{'0': {'Company': 'Amazon Web Services (AWS) ...,"{'0': 'Business Intelligence', '1': 'Software ...",{'0': {'Issue': 'Issued by Ministry of Science...,"{'0': {'URL': 'https://solution.job4u.io/', 'N...",[English],"[VNUHCM - University of Science, HCM Education...","[Business Intelligence, Software Development, ...",1asdfgh,abc1234!,Vietnam,Amazon
2,Hung Hoang,Customer Solutions Manager at Amazon Web Servi...,"Ho Chi Minh City, Vietnam",https://media-exp1.licdn.com/dms/image/C4E03AQ...,Accelerating Cloud adoption…Accelerating Cloud...,"{'0': {'Major': 'Bachelors degree, Computer So...",{'0': {'Company': 'Amazon Web Services (AWS) ...,"{'0': 'Software Project Management', '1': 'Ind...","{'0': {'Issue': 'Issued by FPT Software', 'Hon...",[],"[English, Vietnamese]",[FPT University],"[Software Project Management, Industrial Inter...",2asdfgh,abc1234!,Vietnam,Amazon
3,Robbie Ames,Project Manager at Amazon Web Services (AWS),"Seattle, Washington, United States",https://media-exp1.licdn.com/dms/image/D5635AQ...,"Creative, passionate and upbeat team focused m...","{'0': {'Major': 'BA, Sociology', 'School': 'Un...",{'0': {'Company': 'Amazon Web Services (AWS) ...,"{'0': 'Retail', '1': 'Merchandising', '2': 'In...",[],[],[],[University of Washington],"[Retail, Merchandising, Inventory Management, ...",3asdfgh,abc1234!,United States,Amazon
4,PR B,Project Manager at Amazon web services (AWS),United States,[],[],"{'0': {'Major': 'Bachelors degree, Computer Sc...",{'0': {'Company': 'Amazon Web Services (AWS) ...,{},[],[],[],[New York University],[],4asdfgh,abc1234!,United States,Amazon
5,Andrei Popa,Technical Project Manager at Amazon Web Services,"London, England, United Kingdom",https://media-exp1.licdn.com/dms/image/C4E03AQ...,Technical Program Manager handling all aspects...,[],{'0': {'Company': 'Technical Project Manager a...,"{'0': 'Linux', '1': 'Networking', '2': 'Troubl...",[],[],[],[],"[Linux, Networking, Troubleshooting, Data Cent...",5asdfgh,abc1234!,United Kingdom,Amazon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959,Ramon de Boer,Security Specialist at Microsoft,"Amsterdam-Zuidoost, North Holland, Netherlands",[],[],[],"{'0': {'Company': 'Microsoft', 'Duration': 'Ap...","{'0': 'Security', '1': 'Cloud Computing', '2':...",[],[],[],[],"[Security, Cloud Computing, Network Security, ...",1959asdfgh,abc1234!,Netherlands,Microsoft
1960,Taurayi Kaurayi,CISSP | CISA | OSCP | CEH Master | CCNA CyberO...,"City of Johannesburg, Gauteng, South Africa",https://media-exp1.licdn.com/dms/image/C4D03AQ...,Taurayi Trust Kaurayi is an experienced and qu...,"{'0': {'Major': 'Bachelor of Science (BSc), In...","{'0': {'Company': 'IBM', 'Duration': 'Nov 2018...","{'0': 'VPN', '1': 'CCNA', '2': 'Network Design...",[],[],"[English, Ndebele, North, Shona]",[University of South Africa/Universiteit van S...,"[VPN, CCNA, Network Design, Servers, Routing, ...",1960asdfgh,abc1234!,South Africa,IBM
1961,Kirana Dhiatama Ayunda,"IT Specialist Security | CEH, eJPT, RHCSA, CSCU",Jakarta Metropolitan Area,[],"Hello! I am Kirana, currently working as IT Sp...",{'0': {'Major': 'Bachelor of Computer Informat...,"{'0': {'Company': 'IBM', 'Duration': 'Null', '...","{'0': 'Packet Tracer', '1': 'VMware', '2': 'Pe...",[],"{'0': {'URL': 'Null', 'Name': 'Implementation ...",[],[Telkom University],"[Packet Tracer, VMware, Penetration Testing, P...",1961asdfgh,abc1234!,Jakarta Metropolitan Area,IBM
